## Create Edits that match post edit class distribution of target class to real edits

In [1]:
# General imports
import torch
import numpy as np
import os, sys
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from datetime import datetime

In [3]:
# Local imports
sys.path.insert(0, 'src')
from utils import read_json, read_lists, list_to_dict, ensure_dir, informal_log, write_lists, get_common_dir_path
from utils.df_utils import load_and_preprocess_csv
from utils.visualizations import histogram
from utils.model_utils import prepare_device, quick_predict
from parse_config import ConfigParser
from test import predict, predict_with_bump
import datasets.datasets as module_data
import model.model as module_arch
import model.metric as module_metric
import model.loss as module_loss


In [19]:
# Define constants, paths
class_list_path = os.path.join('metadata', 'cinic-10', 'class_names.txt')

# config_path = 'configs/copies/edit_experiments/cinic10_imagenet_bump_noise.json'
config_path = 'configs/eac/edit_experiments/cinic10_imagenet_bump_corresponding.json'

In [20]:
# Load config file, models, and dataloader
class_list = read_lists(class_list_path)
class_idx_dict = list_to_dict(class_list)

config_dict = read_json(config_path)
config = ConfigParser(config_dict)
device, device_ids = prepare_device(config_dict['n_gpu'])

In [21]:
# Load model
layernum = config.config['layernum']
model = config.init_obj('arch', module_arch, layernum=layernum, device=device)
model.eval()


ModelWrapperSanturkar(
  (model): VGG(
    (normalize): InputNormalize()
    (layer0): Sequential(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer1): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (layer3): Sequential(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.

In [22]:
# Load datasets
# config = ConfigParser(config_dict)
data_loader_args = dict(config.config["data_loader"]["args"])
dataset_args = dict(config["dataset_args"])

val_image_paths = read_lists(config_dict['dataset_paths']['valid_images'])
val_labels = read_lists(config_dict['dataset_paths']['valid_labels'])
val_paths_data_loader = torch.utils.data.DataLoader(
    module_data.CINIC10Dataset(
        data_dir="",
        image_paths=val_image_paths,
        labels=val_labels,
        return_paths=True,
        **dataset_args
    ),
    **data_loader_args
)

# Obtain loss function and metric functions
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

In [31]:
target_class_name = 'automobile'
target_class_idx = class_idx_dict[target_class_name]
n_select = 100
timestamp = '0214_112633'
bumps_timestamp = '0208_112555'
save_timestamp = '0227_144100'

root_dir = os.path.join('saved', 'edit_eac', 'trials', 'CINIC10_ImageNet-VGG_16', '{}' + '_{}'.format(n_select), timestamp)
csv_path_template = os.path.join(root_dir, 'results_table.csv')


### Save class distribution histograms

In [37]:
def save_class_distribution(csv_path, 
                            target_class_idx,
                            target_class_name,
                            show=False,
                            histogram_save_path=None,
                            data_save_path=None):
    
    df = load_and_preprocess_csv(
        csv_path=csv_path,
        drop_duplicates=['ID']
    )
    
    # Obtain number of predictions for target class pre edit
    pre_edit_class_distribution = df['Pre Class Dist'].to_numpy()
    pre_edit_class_distribution = np.stack(pre_edit_class_distribution)
    target_pre_edit_class_predictions = np.mean(pre_edit_class_distribution[:, target_class_idx])
    
    # Obtain number of predictions for target class post edit for each trial
    class_distribution = df['Post Class Dist'].to_numpy()
    class_distribution = np.stack(class_distribution, axis=0)
    target_class_distribution = class_distribution[:, target_class_idx]
    # target_class_bins = np.bincount(target_class_distribution)
    if histogram_save_path is None:
        histogram_save_path = os.path.join(os.path.dirname(csv_path), 'graphs', 'summary', 'target_class_distribution.png')
    title = 'Post Edit {} Class Distribution for {} Edits'.format(target_class_name, target_class_name)
    xlabel = 'Num. {} Predictions Post Edit'.format(target_class_name)
    ylabel = 'Num. Edits'
    
    bin_values, bins, _ = histogram(
        data=target_class_distribution,
        n_bins=50,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        marker=target_pre_edit_class_predictions,
        show=show,
        save_path=histogram_save_path)
    
    bin_tuples = []
    for bin_idx in range(len(bins) - 1):
        bin_tuples.append((bins[bin_idx], bins[bin_idx+1]))

    save_data = {
        "n_target_predictions": target_class_distribution,
        "histogram_bin_values": bin_values,
        "histogram_bins": bins
    }
    
    if data_save_path is None:
        data_save_path = os.path.join(os.path.dirname(csv_path), 'target_class_distribution.pth')
    torch.save(save_data, data_save_path)
    
    print("Saved target class distribution & histogram data to {}".format(data_save_path))
    
    plt.close('all')
    return save_data

#### Loop over all classes and save histograms and distributions

In [38]:
for class_name in class_list:
    csv_path = csv_path_template.format(class_name)
    save_class_distribution(
        csv_path=csv_path,
        target_class_idx=class_idx_dict[class_name],
        target_class_name=class_name,
        show=False,
        data_save_path=os.path.join('saved/edit_eac/experiments/bump_edits/bumps_preds_metrics/{}_{}_bumps_preds_metrics.pth'.format(class_name, n_select)))
    

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Saved target class distribution & histogram data to saved/edit_eac/experiments/bump_edits/bumps_preds_metrics/airplane_100_bumps_preds_metrics.pth
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Saved target class distribution & histogram data to saved/edit_eac/experiments/bump_edits/bumps_preds_metrics/automobile_100_bumps_preds_metrics.pth
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
Saved target class distribution & histogram data to saved/edit_eac/experiments/bump_edits/bumps_preds_metrics/bird_100_bumps_preds_metrics.pth
No artists with labels found to put in legend.  Note that artists whose label start with a

### Get bump amounts on the bin level

In [55]:
# def match_bump_edits(data_loader,
#                      model,
#                      loss_fn,
#                      metric_fns,
#                      device,
#                      bin_lows,
#                      target_class_idx,
#                      save_dir,
#                      debug=True):
#     '''
#     Get bins
#     for each bin, get the mean
#     until the target class predictions on val match the mean,
#         adjust the target
#     store (mean, bump amount)
#     '''
#     # save_dir = os.path.dirname(str(config.save_dir))
#     informal_log_path = os.path.join(save_dir, 'informal_log.txt')
#     bump_dictionary = {}
    
#     n_bins = len(bin_lows) - 1
    
#     n_bins_left = n_bins
#     min_bin = bin_lows[0]
#     bin_width = bin_lows[1] - bin_lows[0]
    
#     bin_means = []
#     bump_amount = 0.2
#     delta = 1.25
    
#     bumps = [-1 for i in range(n_bins)]
#     class_predictions = [-1 for i in range(n_bins)]
#     logs = [None for i in range(n_bins)]
    
#     cur_bump_idx = 0
#     cur_itr = 0
#     delta_reset = 1.25
    
#     # Obtain pre edit metrics
#     pre_edit_log = predict(
#         data_loader=data_loader,
#         model=model,
#         loss_fn=loss_fn,
#         metric_fns=metric_fns,
#         device=device)

#     while cur_bump_idx < n_bins:
#         cur_itr += 1
#         if delta <= 1 or delta > 2:
#             delta = delta_reset
#         if debug:
#             informal_log("[{}]***NEW ITR {}***".format(
#                 datetime.now().strftime(r'%m%d_%H%M%S'), cur_itr), informal_log_path)

#         if bumps[cur_bump_idx] > -1:
#             bump_amount = bumps[cur_bump_idx]
#             cur_bump_idx += 1
            
#             continue
        
#         log = predict_with_bump(
#             data_loader=data_loader,
#             model=model,
#             loss_fn=loss_fn,
#             metric_fns=metric_fns,
#             device=device,
#             target_class_idx=target_class_idx,
#             bump_amount=bump_amount)
        
#         # Obtain num. predictions for target class and determine bin idx
#         post_class_distribution = log['metrics']['predicted_class_distribution']
#         target_class_predictions = post_class_distribution[target_class_idx]
#         bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
        
#         # Check for out of bounds/already visited bin index
#         while bin_idx < cur_bump_idx:
#             bump_amount *= delta
#             if debug:
#                 informal_log("bin_idx < cur_bump_idx ({} and {}) delta={} new bump_amount: {}".format(
#                     bin_idx, cur_bump_idx, delta, bump_amount), informal_log_path)
                
#             log = predict_with_bump(
#                 data_loader=data_loader,
#                 model=model,
#                 loss_fn=loss_fn,
#                 metric_fns=metric_fns,
#                 device=device,
#                 target_class_idx=target_class_idx,
#                 bump_amount=bump_amount)

#             # Obtain num. predictions for target class and determine bin idx
#             post_class_distribution = log['metrics']['predicted_class_distribution']
#             target_class_predictions = post_class_distribution[target_class_idx]
#             bin_idx = math.floor((target_class_predictions - min_bin) / bin_width)
                
            
#             # continue
#         if bin_idx >= n_bins:
#             if cur_bump_idx == 0:
#                 bump_amount /= delta
#             else:
#                 bump_amount = (bump_amount + bumps[cur_bump_idx - 1]) / 2
#             informal_log("bin_idx > n_bins ({}) delta={} new bump_amount: {}".format(
#                 bin_idx, delta, bump_amount), informal_log_path)
#             continue
            
#         # Sanity check correct bin_idx
#         assert target_class_predictions >= bin_lows[bin_idx] and \
#                target_class_predictions <= bin_lows[bin_idx+1]
        
#         # If bump value for this bin yet, assign it
#         if bumps[bin_idx] == -1:
#             bumps[bin_idx] = bump_amount
#             class_predictions[bin_idx] = target_class_predictions
#             logs[bin_idx] = log
#             if debug:
#                 informal_log("assigned bin_idx {}".format(bin_idx), informal_log_path)
#                 informal_log("Cur bump_idx: {} bump amount: {} target_class_pred: {} bin_idx: {}".format(
#                     cur_bump_idx, bump_amount, target_class_predictions, bin_idx), informal_log_path)
        
#         # Check if this is our current bin we're trying to fill
#         if bin_idx == cur_bump_idx:
#             cur_bump_idx += 1
#             bump_amount *= delta
#             if debug:
#                 informal_log("updating cur_bump_idx to {}, bump_amount to {}"
#                              .format(cur_bump_idx, bump_amount), informal_log_path)
            
#         # If we're not at the current cell,
#         else:
#             # Decrease bump amount by half
#             if cur_bump_idx == 0:
#                 bump_amount /= 2
#             # Set bump amount to something in the middle between current and last successful
#             else:
#                 bump_amount = (bump_amount + bumps[cur_bump_idx-1]) / 2 
#             if debug:
#                 informal_log("Overshot to bin {}. Decreasing bump amount to {} aiming for bin {}".format(
#                     bin_idx, bump_amount, cur_bump_idx), informal_log_path)
#         informal_log("bumps: {}".format(bumps), informal_log_path)

    
    
#     save_data = {
#         'pre_edit_metrics': pre_edit_log,
#         'bump_amounts': bumps,
#         'target_class_predictions': class_predictions,
#         'metrics': logs
#     }
#     save_path = os.path.join(save_dir, 'bumps_preds_metrics.pth'.format())
#     torch.save(save_data, save_path)
#     if debug:
#         informal_log("Saved data to {}".format(save_path), informal_log_path)
#     return save_data

In [57]:

# for target_class_idx, target_class_name in enumerate(class_list):
#     if target_class_name == 'airplane' or target_class_name == 'automobile' or target_class_name == 'cat':
#         continue
#     target_class_distribution_path = os.path.join(root_dir, 'target_class_distribution.pth').format(
#         target_class_name)
#     target_class_distribution = torch.load(target_class_distribution_path)
#     bin_lows = target_class_distribution['histogram_bins']

#     bump_save_dir = os.path.join(os.path.dirname(os.path.dirname(config.save_dir)), save_timestamp, '{}_{}'.format(target_class_name, n_select))
#     ensure_dir(bump_save_dir)
#     # Save a copy of histogram info to save_dir
#     torch.save(target_class_distribution, os.path.join(bump_save_dir, 'target_class_distribution.pth'))
#     # Run bump experiments
#     print("Obtaining class distribution for {} from {}".format(target_class_name, target_class_distribution_path))
#     print("Saving results to {}".format(bump_save_dir))
#     bumps_preds_metrics = match_bump_edits(
#         data_loader=val_paths_data_loader,
#         model=model,
#         loss_fn=loss_fn,
#         metric_fns=metric_fns,
#         device=device,
#         bin_lows=bin_lows,
#         target_class_idx=target_class_idx,
#         save_dir=bump_save_dir)

#     # torch.save(bumps_preds_metrics, bumps_preds_save_path)



Obtaining class distribution for bird from saved/edit_eac/trials/CINIC10_ImageNet-VGG_16/bird_100/0214_112633/target_class_distribution.pth
Saving results to saved/edit_eac/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0227_144100/bird_100


100%|██████████████████████████████████████████████████████| 274/274 [00:08<00:00, 33.13it/s]


[0228_091557]***NEW ITR 1***


100%|██████████████████████████████████████████████████████| 274/274 [00:07<00:00, 37.54it/s]


bin_idx < cur_bump_idx (-1 and 0) delta=1.25 new bump_amount: 0.25


...

100%|██████████████████████████████████████████████████████| 274/274 [00:07<00:00, 36.46it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


bin_idx > n_bins (50) delta=1.25 new bump_amount: 6.61224865314259
[0228_093110]***NEW ITR 124***


100%|██████████████████████████████████████████████████████| 274/274 [00:09<00:00, 28.16it/s]
/n/fs/ac-editing/model-editing/src/model/metric.py:168: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(TPs / (TPs + FPs))
/n/fs/ac-editing/model-editing/src/model/metric.py:205: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(2 * precisions * recalls / (precisions + recalls))


assigned bin_idx 49
Cur bump_idx: 49 bump amount: 6.61224865314259 target_class_pred: 52256 bin_idx: 49
updating cur_bump_idx to 50, bump_amount to 8.265310816428237
bumps: [0.762939453125, 1.1920928955078125, 1.862645149230957, 2.0954757928848267, 2.3283064365386963, 2.6193447411060333, 2.9103830456733704, 3.092281986027956, 3.2741809263825417, 3.4788172342814505, 3.637978807091713, 3.751665644813329, 3.865352482534945, 4.092726157978177, 4.220623850414995, 4.348521542851813, 4.547473508864641, 4.604316927725449, 4.7482018317168695, 4.89208673570829, 5.044964446199174, 5.115907697472721, 5.17985654369113, 5.341727060681478, 5.425191546004626, 5.503597577671826, 5.546594433747387, 5.589591289822948, 5.63325997177469, 5.655264893539435, 5.675585001974071, 5.721279658833669, 5.743628407500989, 5.7553961596568115, 5.80036019215413, 5.827338611652522, 5.847572426276315, 5.900069648172222, 6.009442943266663, 6.115108419635362, 6.191547274880804, 6.288290201050817, 6.337417468246526, 6.38692

NameError: name 'bumps_preds_save_path' is not defined

### Perform Bump for a Corresponding Edit

In [62]:
def match_bump(n_target_predictions,
               target_class_idx,
               bumps_preds_metrics,
               results_save_dir,
               data_loader,
               model,
               loss_fn,
               metric_fns,
               device,
               cushion=5,
               n_stop=10,
               debug=True):
    '''
    Given a number of predictions for target class, obtain bump amount to match and save post edit metrics in results_save_dir
    
    Arg(s):
        n_target_predictions : int
            Number of predictions to obtain for target class
        target_class_idx : int
            index of target class
        bumps_preds_metrics : dict
            saved data from match_bump_edits()
        results_save_dir : str
            directory to save results to
        data_loader : torch.utils.data.DataLoader
            validation data loader to obtain metrics for
        model : torch.nn.Module
            model
        loss_fn : module
            loss function
        metric_fns : list[model.metric modules]
            list of metric functions
        device : torch.device
            GPU device to run model on
        cushion : int
            how far away cur_n_target_predictions can be from n_target_predictions on either side to break loop (buffer)
        n_stop : int
            how many iterations when stuck at the same cur_n_target_predictions until to break the loop
        debug : bool
            control verbosity
        
            
    Returns: 
        None
    '''
    # Unpack bumps_preds_metrics
    pre_edit_metrics = bumps_preds_metrics['pre_edit_metrics']
    
    target_class_predictions = bumps_preds_metrics['target_class_predictions']
    bump_amounts = bumps_preds_metrics['bump_amounts']
    
    # Obtain number of total predictions and assert n_target_predictions is less
    try:
        n_predictions_total = np.sum(pre_edit_metrics['metrics']['predicted_class_distribution'])
    except:
        n_predictions_total = np.sum(pre_edit_metrics['predicted_class_distribution'])
    assert n_target_predictions <= n_predictions_total, \
        "n_target_predictions ({}) must be less than total number of data in dataloader ({})".format(n_target_predictions, n_predictions_total)
    
    # Find index above and below n_target_predictions
    bin_high_idx = -1
    for bin_idx, target_class_prediction in enumerate(target_class_predictions):
        if target_class_prediction > n_target_predictions:
            bin_high_idx = bin_idx
            break
            
    if bin_high_idx == -1: # Past upper end 
        # n_predictions_upper_bound
        bump_amount_upper_bound = bump_amounts[bin_high_idx] * 2
        bump_amount_lower_bound = bump_amounts[bin_high_idx]
    # Fall into a bin from the histogram (or lower)
    else: 
        n_predictions_upper_bound = target_class_predictions[bin_high_idx]
        bump_amount_upper_bound = bump_amounts[bin_high_idx]

        # Store lower bounds for bump_amount and n_predictions
        bin_low_idx = bin_high_idx - 1
        if bin_low_idx > -1: # First bin is already higher than n_target_predictions
            n_predictions_lower_bound = target_class_predictions[bin_low_idx]
            bump_amount_lower_bound = bump_amounts[bin_low_idx]
        else: # n_target_predictions is less than bump amount for the first bin
            n_predictions_lower_bound = 0
            bump_amount_lower_bound = -10 
        
    cur_n_target_predictions = 0
    if debug:
        print("target n_predictions: {}".format(n_target_predictions))
        print("Initial bounds for bump: ({}, {})".format(bump_amount_lower_bound, bump_amount_upper_bound))
    
    # Keep looping while the difference between current n_target_predictions and goal n_target_predictions is too large
    while abs(cur_n_target_predictions - n_target_predictions) > cushion:
        # Update bump amount
        cur_bump_amount = (bump_amount_lower_bound + bump_amount_upper_bound) / 2
        # Check before we undergo an infinite loop
        if cur_bump_amount == 0:
            print("cur_bump_amount is 0, exiting loop")
            break
            
        # predict using logit bump
        log = predict_with_bump(
                data_loader=data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=cur_bump_amount,
                output_save_path=os.path.join(results_save_dir, "post_edit_logits.pth"),
                log_save_path=os.path.join(results_save_dir, "post_edit_metrics.pth"))

        # Obtain num. predictions for target class and determine bin idx
        post_class_distribution = log['metrics']['predicted_class_distribution']
        cur_n_target_predictions = post_class_distribution[target_class_idx]
        if debug:
            print("cur_bump_amount: {}, cur_n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
        
        # Update bump bounds of binary search
        if cur_n_target_predictions > n_target_predictions:
            bump_amount_upper_bound = cur_bump_amount
            if debug:
                print("Updated upper bound to {}".format(bump_amount_upper_bound))
        elif cur_n_target_predictions < n_target_predictions:
            bump_amount_lower_bound = cur_bump_amount
            if debug:
                print("Updated lower bound to {}".format(bump_amount_lower_bound))
        
    if debug:
        print("final results: bump amount: {} n_target_predictions: {}".format(cur_bump_amount, cur_n_target_predictions))
    # if results_save_dir is not None:
    #     torch.save(log, os.path.join(results_save_dir, "post_edit_metrics.pth"))
        # torch.save(logits, os.path.join(results_save_dir, "post_edit_logits.pth"))
    return cur_bump_amount, cur_n_target_predictions, log
    

In [52]:
# # testing

# bumps_preds_metrics_path = os.path.join(
#     "saved/edit/experiments/bump_edits",
#     "bumps_preds_metrics",
#     "{}_{}_bumps_preds_metrics.pth".format(target_class_name, n_select))
# bumps_preds_metrics = torch.load(bumps_preds_metrics_path)

# n_target_predictions = 65000
# results_save_dir = os.path.join('temp')
# match_bump(n_target_predictions=n_target_predictions,
#            target_class_idx=target_class_idx,
#            bumps_preds_metrics=bumps_preds_metrics,
#            results_save_dir=results_save_dir,
#            data_loader=val_paths_data_loader,
#            model=model,
#            loss_fn=loss_fn,
#            metric_fns=metric_fns,
#            device=device,
#            cushion=5,
#            debug=True)

In [63]:
n_edits = 5
cushion = 10
for target_class_idx, target_class_name in enumerate(class_list):
    # Already did airplane,  automobile, bird class
    if target_class_idx <= 2:
        continue
        
    original_trials_trial_paths_path = os.path.join(root_dir.format(target_class_name), 'trial_paths.txt')
    original_trial_paths = read_lists(original_trials_trial_paths_path)
    common_path = get_common_dir_path(original_trial_paths)
    print("Length of trial paths: {}".format(len(original_trial_paths)))

    # Data structure to store how much to bump for each n_predictions in target class
    bump_amount_dictionary_path = os.path.join(
        'metadata',
        config_dict['name'], # CINIC10_ImageNet-VGG16
        'bump_amounts',
        '{}_{}'.format(target_class_name, n_select),
        'logit_bump_buffer_{}_dict.pth'.format(cushion))

    if os.path.isfile(bump_amount_dictionary_path):
        bump_amount_dictionary = torch.load(bump_amount_dictionary_path)
    else:
        bump_amount_dictionary = {}
    ensure_dir(os.path.dirname(bump_amount_dictionary_path))


    # Store histogram information
    bumps_preds_metrics_path = os.path.join(
        "saved/edit_eac/experiments/bump_edits",
        "bumps_preds_metrics",
        "{}_{}_bumps_preds_metrics.pth".format(target_class_name, n_select))
    bumps_preds_metrics = torch.load(bumps_preds_metrics_path)

    # Create directories and paths
    result_root = os.path.join(
            os.path.dirname(os.path.dirname(config.save_dir)), 
            save_timestamp, 
            '{}_{}'.format(target_class_name, n_select))
    ensure_dir(result_root)
    progress_report_path = os.path.join(result_root, 'progress_reports.txt')
    # Create file to store paths
    logit_bump_trial_paths_path = os.path.join(result_root, 'trial_paths.txt')
    if os.path.isfile(logit_bump_trial_paths_path):
        os.remove(logit_bump_trial_paths_path)

    # Obtain pre edit metrics
    informal_log("Target class: {}".format(target_class_name), progress_report_path)
    informal_log("Logging to {}".format(progress_report_path), progress_report_path)
    informal_log("Calculating pre-bumping stats...", progress_report_path)
    # Store pre-edit metrics
    pre_edit_log = predict_with_bump(
        data_loader=val_paths_data_loader,
        model=model,
        loss_fn=loss_fn,
        metric_fns=metric_fns,
        device=device,
        target_class_idx=target_class_idx,
        bump_amount=0)

    # Iterate through all trial paths
    for trial_idx, trial_path in enumerate(original_trial_paths):
        # if trial_idx <= 151:
        #     continue
        # Obtain trial ID and create save directory for logit bump results
        trial_id = trial_path[len(common_path)+1:]
        logit_bump_trial_save_dir = os.path.join(
            result_root,
            'results',
            trial_id,
            'models')
        print("Save dir: {}".format(logit_bump_trial_save_dir))
        ensure_dir(logit_bump_trial_save_dir)

        # Obtain desired n_target_predictions
        trial_post_edit_metrics_path = os.path.join(
            trial_path,
            'models',
            'post_edit_metrics.pth')
        trial_post_edit_metrics = torch.load(trial_post_edit_metrics_path)
        n_target_predictions = trial_post_edit_metrics['predicted_class_distribution'][target_class_idx]

        # Save pre bump metrics and logits in directory
        torch.save(pre_edit_log['metrics'], os.path.join(logit_bump_trial_save_dir, "pre_edit_metrics.pth"))
        torch.save(pre_edit_log['logits'], os.path.join(logit_bump_trial_save_dir, "pre_edit_logits.pth"))

        # If bump amount is in dictionary, obtain that value
        if n_target_predictions in bump_amount_dictionary:

            informal_log("[{}] Found corresponding logit bump for {} in dictionary ({}/{})".format(
                datetime.now().strftime(r'%m%d_%H%M%S'),
                trial_id,
                trial_idx + 1,
                len(original_trial_paths)), progress_report_path)
            bump_amount = bump_amount_dictionary[n_target_predictions]

            log = predict_with_bump(
                data_loader=val_paths_data_loader,
                model=model,
                loss_fn=loss_fn,
                metric_fns=metric_fns,
                device=device,
                target_class_idx=target_class_idx,
                bump_amount=bump_amount,
                output_save_path=os.path.join(logit_bump_trial_save_dir, "post_edit_logits.pth"),
                log_save_path=os.path.join(logit_bump_trial_save_dir, "post_edit_metrics.pth"))
            # Sanity check that the n_target_predictions with bump is within buffer of target
            bump_n_target_predictions = log['metrics']['predicted_class_distribution'][target_class_idx]
            assert abs(n_target_predictions - bump_n_target_predictions) <= cushion


        else:
            informal_log("[{}] Creating corresponding logit bump for {} ({}/{})".format(
                datetime.now().strftime(r'%m%d_%H%M%S'),
                trial_id,
                trial_idx + 1,
                len(original_trial_paths)), progress_report_path)
            # Find corresponding bump amount
            bump_amount, n_target_predictions, metrics = \
                match_bump(
                    n_target_predictions=n_target_predictions,
                    target_class_idx=target_class_idx,
                    bumps_preds_metrics=bumps_preds_metrics,
                    results_save_dir=logit_bump_trial_save_dir,
                    data_loader=val_paths_data_loader,
                    model=model,
                    loss_fn=loss_fn,
                    metric_fns=metric_fns,
                    device=device,
                    cushion=cushion,
                    debug=True)
            bump_amount_dictionary[n_target_predictions] = bump_amount

        # Write trial path to list
        informal_log(os.path.dirname(logit_bump_trial_save_dir), logit_bump_trial_paths_path)

        # Update the bump dictionary on file
        if trial_idx % 10 == 0:
            torch.save(bump_amount_dictionary, bump_amount_dictionary_path)

Length of trial paths: 128
Target class: cat
Logging to saved/edit_eac/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0227_144100/cat_100/progress_reports.txt
Calculating pre-bumping stats...


100%|██████████████████████████████████████████████████████| 274/274 [00:11<00:00, 24.25it/s]


Save dir: saved/edit_eac/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0227_144100/cat_100/results/cat-train-n02127052_1053/felzenszwalb_masked_softmax/models
[0228_145851] Creating corresponding logit bump for cat-train-n02127052_1053/felzenszwalb_masked_softmax (1/128)
target n_predictions: 18232
Initial bounds for bump: (3.696243311424041, 3.8117509149060425)


100%|██████████████████████████████████████████████████████| 274/274 [00:07<00:00, 37.18it/s]


cur_bump_amount: 3.753997113165042, cur_n_target_predictions: 18320
Updated upper bound to 3.753997113165042


...

cur_bump_amount: 3.115019353572279, cur_n_target_predictions: 11327
Updated upper bound to 3.115019353572279


100%|██████████████████████████████████████████████████████| 274/274 [00:07<00:00, 35.97it/s]


cur_bump_amount: 3.1036506698001176, cur_n_target_predictions: 11285
Updated upper bound to 3.1036506698001176


100%|██████████████████████████████████████████████████████| 274/274 [00:07<00:00, 36.03it/s]

cur_bump_amount: 3.097966327914037, cur_n_target_predictions: 11270
Updated lower bound to 3.097966327914037
final results: bump amount: 3.097966327914037 n_target_predictions: 11270
saved/edit_eac/experiments/corresponding_bump_edits/CINIC10_ImageNet-VGG_16/0227_144100/truck_100/results/truck-train-n03345487_17869/felzenszwalb_gaussian_softmax


In [14]:
ensure_dir(os.path.dirname(bump_amount_dictionary_path))
torch.save(bump_amount_dictionary, bump_amount_dictionary_path)